<a href="https://colab.research.google.com/github/jenny005/HuggingFace_Tutorial/blob/main/HF_Summaration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------- Install Dependencies --------------------
!pip install -q transformers langchain langchain-huggingface accelerate ipywidgets

In [ ]:
# -------------------- Environment Setup --------------------
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Helps trace CUDA errors

# -------------------- Imports --------------------
import torch
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers.utils.logging import set_verbosity_error
import IPython.display as display
import ipywidgets as widgets

# Suppress Hugging Face warnings
set_verbosity_error()

# -------------------- Device Setup --------------------
device = 0 if torch.cuda.is_available() else -1
print("✅ Using:", torch.cuda.get_device_name(0) if device == 0 else "CPU")

# -------------------- Define Pipelines --------------------
try:
    summarization_pipeline = pipeline(
        "summarization",
        model="facebook/bart-large-cnn",
        tokenizer="facebook/bart-large-cnn",
        device=device,
        torch_dtype=torch.float32
    )
    summarizer = HuggingFacePipeline(pipeline=summarization_pipeline)

    refinement_pipeline = pipeline(
        "summarization",
        model="facebook/bart-large",
        tokenizer="facebook/bart-large",
        device=device,
        torch_dtype=torch.float32
    )
    refiner = HuggingFacePipeline(pipeline=refinement_pipeline)

    qa_pipeline = pipeline(
        "question-answering",
        model="deepset/roberta-base-squad2",
        tokenizer="deepset/roberta-base-squad2",
        device=device,
        torch_dtype=torch.float32
    )

except RuntimeError as e:
    print("❌ CUDA error, falling back to CPU.")
    device = -1

    summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)
    summarizer = HuggingFacePipeline(pipeline=summarization_pipeline)

    refinement_pipeline = pipeline("summarization", model="facebook/bart-large", device=-1)
    refiner = HuggingFacePipeline(pipeline=refinement_pipeline)

    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", device=-1)

# -------------------- LangChain Summary Template --------------------
summary_template = PromptTemplate.from_template("Summarize the following text in a {length} way:\n\n{text}")
summarization_chain = summary_template | summarizer | refiner

# -------------------- Colab UI --------------------
text_widget = widgets.Textarea(
    value="Paste your text here...",
    placeholder="Enter text to summarize...",
    description="Input Text:",
    layout=widgets.Layout(width="100%", height="150px")
)

length_widget = widgets.Dropdown(
    options=["short", "medium", "long"],
    value="medium",
    description="Summary Length:"
)

button = widgets.Button(description="Generate Summary", button_style='success')
output = widgets.Output()

def generate_summary(_):
    output.clear_output()
    with output:
        text = text_widget.value.strip()[:2048]  # Prevent out-of-memory errors
        length = length_widget.value
        if not text:
            print("❌ Please enter some text.")
            return

        print("⏳ Generating summary...")
        try:
            summary = summarization_chain.invoke({"text": text, "length": length})
            print("\n🔹 **Generated Summary:**\n")
            print(summary)

            # --- QA UI with Persistent History ---
            print("\nAsk a question about the summary 👇")

            qa_input = widgets.Text(
                value='',
                placeholder='Ask a question...',
                description='QA:',
                layout=widgets.Layout(width='100%')
            )

            qa_submit = widgets.Button(description="Submit Question", button_style='info')
            qa_output = widgets.Output()

            def on_qa_submit(b):
                question = qa_input.value.strip()
                if not question or question.lower() == "exit":
                    return
                result = qa_pipeline(question=question, context=summary)
                with qa_output:
                    print(f"❓ {question}")
                    print(f"🔹 Answer: {result['answer']}\n")
                qa_input.value = ''

            qa_submit.on_click(on_qa_submit)
            display.display(widgets.VBox([qa_input, qa_submit, qa_output]))

        except Exception as e:
            print("❌ Error during processing:")
            print(e)

# Bind button
button.on_click(generate_summary)

# Show UI
display.display(text_widget, length_widget, button, output)


✅ Using: NVIDIA L4


Textarea(value='Paste your text here...', description='Input Text:', layout=Layout(height='150px', width='100%…

Dropdown(description='Summary Length:', index=1, options=('short', 'medium', 'long'), value='medium')

Button(button_style='success', description='Generate Summary', style=ButtonStyle())

Output()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
